In [ ]:
# =========================================
# 0) 설치 (Colab)
# =========================================
!pip install -q transformers datasets accelerate

# =========================================
# 1) 임포트
# =========================================
import os, torch
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForCausalLM,
    TrainingArguments, Trainer
)

# (선택) CUDA 에러 위치 상세 확인: 필요시 주석 해제
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"

# =========================================
# 2) 모델/토크나이저 로딩 + PAD/EOS 정렬 + 임베딩 리사이즈
# =========================================
model_name = "skt/kogpt2-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# PAD를 EOS로 통일
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name)
model.config.pad_token_id = tokenizer.eos_token_id

# ★ 토크나이저 길이와 임베딩 테이블 동기화
#   통계 기반 초기화(mean_resizing)를 꺼서 디버깅 단순화
model.resize_token_embeddings(len(tokenizer), mean_resizing=False)

# =========================================
# 3) 초소형 학습 데이터
# =========================================
examples = [
{"instruction": "안녕", "input": "", "output": "안녕! 나는 집수리 도우미야!"},
{"instruction": "누구야?", "input": "", "output": "나는 집수리 전문가 챗봇이야. 집 고치는 거 물어봐!"},
{"instruction": "싱크대 물이 새", "input": "", "output": "배수관 연결부를 확인해봐. 실리콘 마감이 오래됐을 수도 있어."},
{"instruction": "화장실 곰팡이 어떻게 없애?", "input": "", "output": "락스 희석액을 분무기에 담아서 뿌리고 환기 잘 시켜!"},
{"instruction": "벽에 금이 갔어", "input": "", "output": "구조 금이면 전문가 부르고, 표면 금이면 퍼티로 메꾸고 도색하면 돼."},
{"instruction": "도배 직접 할 수 있을까?", "input": "", "output": "가능하지! 하지만 초보면 셀프시트지나 부분도배부터 연습해봐."},
{"instruction": "장판 교체 어려워?", "input": "", "output": "초보도 할 수 있어. 다만 바닥 평탄작업이 제일 중요해!"},
{"instruction": "에어컨에서 냄새 나", "input": "", "output": "필터 청소하고 내부 살균세척 해봐. 곰팡이 때문일 가능성이 커."},
{"instruction": "전등이 깜빡거려", "input": "", "output": "형광등이면 안정기 문제일 수 있고, LED면 접촉불량일 수 있어."},
{"instruction": "싱크대 배수구 냄새 올라와", "input": "", "output": "트랩에 물이 말랐을 수도 있어. 물을 자주 흘려보내거나 트랩 교체해봐."},
{"instruction": "현관문이 잘 안 닫혀", "input": "", "output": "경첩이나 도어클로저 나사를 조정해봐. 그래도 안 되면 문틀이 틀어진 걸 수도 있어."},
{"instruction": "도어클로저 조정법 알려줘", "input": "", "output": "나사 두 개 중 속도 조절 나사를 살짝 돌려서 닫히는 속도를 맞추면 돼."},
{"instruction": "보일러가 안 켜져", "input": "", "output": "가스 밸브, 전원, 온도조절기 전원 확인 먼저 해봐."},
{"instruction": "보일러 소리 심해", "input": "", "output": "난방수에 공기 들어갔을 수 있어. 에어벤트로 공기 빼줘야 해."},
{"instruction": "도배 전에 뭘 준비해야 해?", "input": "", "output": "벽면 청소, 퍼티로 구멍 메우기, 초배지 붙이기 순서로 하면 좋아."},
{"instruction": "페인트칠 잘하는 팁 좀", "input": "", "output": "붓보단 롤러로 일정하게, 두껍게 한 번보다 얇게 두세 번이 좋아."},
{"instruction": "곰팡이 방지 페인트 효과 있어?", "input": "", "output": "있어! 곰팡이 방지 성분이 들어 있어서 습한 곳에 유용해."},
{"instruction": "욕실 실리콘 곰팡이 제거법", "input": "", "output": "락스 젤을 올려두고 1시간 뒤 닦으면 돼. 심하면 새로 실리콘 쏴야 해."},
{"instruction": "창문 틈새 바람 막는 법", "input": "", "output": "문풍지 붙이거나 실리콘으로 틈을 메워봐."},
{"instruction": "전기 콘센트에서 스파크 났어", "input": "", "output": "즉시 전원 차단하고 전문가 불러! 위험할 수 있어."},
{"instruction": "누전 차단기가 자주 내려가", "input": "", "output": "누전 가능성이 있어. 가전제품 하나씩 빼서 원인을 찾아봐."},
{"instruction": "천장에서 물 새", "input": "", "output": "옥상 방수나 배관 문제야. 임시로 물받이 두고 바로 관리사무소 연락해."},
{"instruction": "벽지에 얼룩 생겼어", "input": "", "output": "습기나 담배연기일 수도 있어. 베이킹소다 물로 닦아봐."},
{"instruction": "싱크대 문이 덜컥거려", "input": "", "output": "힌지 나사가 풀렸을 확률이 높아. 드라이버로 조여줘."},
{"instruction": "창문이 잘 안 열려", "input": "", "output": "레일에 먼지나 모래 낀 거야. 진공청소기로 청소하고 실리콘 윤활제 뿌려봐."},
{"instruction": "욕실 타일 떨어졌어", "input": "", "output": "본드나 시멘트풀로 다시 붙이되, 방수층 손상은 없는지 확인해봐."},
{"instruction": "싱크대 물이 잘 안내려가", "input": "", "output": "배수구 트랩에 음식물 찌꺼기 낀 거야. 뜨거운 물과 베이킹소다로 녹여봐."},
{"instruction": "세탁기 호스가 새", "input": "", "output": "연결부 고무패킹이 닳았을 가능성이 커. 새 호스로 교체해."},
{"instruction": "현관 바닥 타일 깨졌어", "input": "", "output": "깨진 부분만 교체 가능해. 동일 타일 구해서 줄눈까지 다시 시공해봐."},
{"instruction": "천장에 곰팡이 생겼어", "input": "", "output": "결로 때문일 수 있어. 단열보강하거나 제습기 사용해봐."},
{"instruction": "도배랑 페인트 중 뭐가 나아?", "input": "", "output": "도배는 깔끔하고 빠르지만 페인트는 색감 다양하고 관리 쉬워!"}
]
ds = Dataset.from_list(examples)

# =========================================
# 4) 전처리: 프롬프트/라벨 분리 + 프롬프트 마스킹 + EOS 부여
# =========================================
def to_sample(e):
    prompt = "질문: " + e["instruction"] + " " + e["input"] + "\n답변:"
    target = e["output"] + tokenizer.eos_token  # ★ 종료 학습(EOS)

    enc_p = tokenizer(prompt, truncation=True, max_length=128, add_special_tokens=False)
    enc_t = tokenizer(target, truncation=True, max_length=128, add_special_tokens=False)

    input_ids = enc_p["input_ids"] + enc_t["input_ids"]
    attention_mask = [1] * len(input_ids)

    # 프롬프트는 -100(손실 제외), 타깃 토큰만 학습
    labels = [-100] * len(enc_p["input_ids"]) + enc_t["input_ids"]

    return {"input_ids": input_ids, "attention_mask": attention_mask, "labels": labels}

tokenized = ds.map(to_sample, remove_columns=ds.column_names)

# =========================================
# 5) vocab 범위 검사: 임베딩 인덱스 초과 사전 차단
# =========================================
vocab_size = model.get_input_embeddings().weight.size(0)

def assert_in_range(ex):
    for k in ("input_ids", "labels"):
        if k in ex:
            ids = [t for t in ex[k] if t >= 0]  # -100은 라벨 마스크라 제외
            if ids and max(ids) >= vocab_size:
                raise ValueError(k + "에 vocab_size 초과 토큰: " + str(max(ids)) + " >= " + str(vocab_size))
    return ex

_ = tokenized.map(assert_in_range)

# =========================================
# 6) 커스텀 Collator: 가변 길이 패딩 (labels는 -100으로 패딩)
# =========================================
pad_id = tokenizer.pad_token_id

def causal_lm_collator(features):
    max_len = max(len(f["input_ids"]) for f in features)
    input_ids, attention_mask, labels = [], [], []

    for f in features:
        L = len(f["input_ids"])
        pad_len = max_len - L

        input_ids.append(f["input_ids"] + [pad_id] * pad_len)
        attention_mask.append(f["attention_mask"] + [0] * pad_len)
        labels.append(f["labels"] + [-100] * pad_len)

    batch = {
        "input_ids": torch.tensor(input_ids, dtype=torch.long),
        "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
        "labels": torch.tensor(labels, dtype=torch.long),
    }
    return batch

collator = causal_lm_collator

# =========================================
# 7) 학습 인자 (에폭 기준, max_steps 미사용)
# =========================================
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./kogpt2-ultralight",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=2,
    num_train_epochs=10,      # 소규모 데이터면 10~20도 가능
    learning_rate=2e-5,
    warmup_ratio=0.1,
    weight_decay=0.01,
    save_strategy="no",
    logging_steps=10,
    report_to="none",
    fp16=False                # 먼저 False로 안정 확인 후 True로 변경 가능
)

# =========================================
# 8) 트레이너 & 학습
# =========================================
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized,
    data_collator=collator
)

trainer.train()

# =========================================
# 9) 저장
# =========================================
save_dir = "./kogpt2-ultralight"
trainer.save_model(save_dir)
tokenizer.save_pretrained(save_dir)

# =========================================
# 10) 생성 함수 (반복 억제 + EOS로 종료)
# =========================================
from transformers import AutoTokenizer, AutoModelForCausalLM

tok = AutoTokenizer.from_pretrained(save_dir)
mdl = AutoModelForCausalLM.from_pretrained(save_dir)

tok.pad_token = tok.eos_token
mdl.config.pad_token_id = tok.eos_token_id

if torch.cuda.is_available():
    mdl.cuda()

def chat_once(q):
    prompt = "질문: " + q + "\n답변:"
    inputs = tok(prompt, return_tensors="pt")
    if torch.cuda.is_available():
        inputs = {k: v.cuda() for k, v in inputs.items()}

    output_ids = mdl.generate(
        **inputs,
        max_new_tokens=80,
        do_sample=True,
        temperature=0.7,        # 산만함 조절
        top_p=0.9,
        top_k=50,
        repetition_penalty=1.3, # 반복 억제
        no_repeat_ngram_size=3, # n-그램 반복 금지
        eos_token_id=tok.eos_token_id,
        pad_token_id=tok.eos_token_id
    )

    text = tok.decode(output_ids[0], skip_special_tokens=True)
    if "답변:" in text:
        return text.split("답변:")[-1].strip()
    return text.strip()

# =========================================
# 11) 간단 대화 루프
# =========================================
print("초경량 챗봇 시작 (종료 입력 시 끝)\n")
while True:
    q = input("사용자: ")
    if q.strip() == "종료":
        break
    print("🤖:", chat_once(q))


Map:   0%|          | 0/31 [00:00<?, ? examples/s]

Map:   0%|          | 0/31 [00:00<?, ? examples/s]

`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
10,5.113200
20,3.262900
30,1.842200
40,1.140300
50,0.710600
60,0.366600
70,0.290600
